### **Setup** 

In [19]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()

WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
print(f"Weather API Key: {WEATHER_API_KEY}")

Weather API Key: af1c9d2fbd3843f704c67e24929dc95a


### **Financial agent**


In [26]:
def get_indicator(indicator: str):
   """
   Get financial information.
   This function fetches data from the Mindicator API for a given indicator.
   It constructs the URL using the indicator name and makes a GET request to the API.
   The response is expected to be in JSON format, and the function returns the data if the request is successful.
   If the request fails, an exception is raised with an error message.
      
      Example:
         get_indicator("dolar")
      Example:
         get_indicator("uf")
      
      Params  
         indicator (str): The indicator to fetch.
      Returns:
         dict: The data for the indicator.
   """
   url = f"https://mindicator.cl/api/{indicator}"
   data = requests.get(url).json()
   if data["success"]:
      return data["data"]
   else:
      raise Exception(f"Error fetching indicator {indicator}: {data['message']}")  

In [27]:
from langchain_core.prompts import PromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.tools import StructuredTool
from langchain.agents import (create_react_agent, AgentExecutor)

financial_llm = OllamaLLM(model="llama3", base_url="http://localhost:11434", temperature=0)

template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question


Begin!

Question: {input}
Thought: {agent_scratchpad}
"""

prompt = PromptTemplate(
   template=template,
   input_variables=[
      "input", 
      "tools", 
      "tool_names", 
      "agent_scratchpad"
   ], 
)

tools_for_agent = [
   StructuredTool.from_function(
      name="get_indicator",
      func=get_indicator,
      description="""Get financial information.
   This function fetches data from the Mindicator API for a given indicator.
   It constructs the URL using the indicator name and makes a GET request to the API.
   The response is expected to be in JSON format, and the function returns the data if the request is successful.
   If the request fails, an exception is raised with an error message.
      
      Example:
         get_indicator("dolar")
      Example:
         get_indicator("uf")
      
      Params  
         indicator (str): The indicator to fetch.
      Returns:
         dict: The data for the indicator.
     """,
   )
]

financial_agent = create_react_agent(
   llm=financial_llm, 
   tools=tools_for_agent,
   prompt=prompt
)

financial_agent_executor = AgentExecutor(
   agent=financial_agent, 
   tools=tools_for_agent, 
   verbose=True,
)

### **Tavily search**

In [43]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Web search
def get_location(location):
   """
   Get the location from the user.
   """
   search = TavilySearchResults(max_results=3)
   res = search.run(location)
   return res

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.tools import StructuredTool
from langchain.agents import (create_react_agent, AgentExecutor)

template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

For weather questions about common cities, use these coordinates:
- Madrid: lat=40.4168, lon=-3.7038
- Barcelona: lat=41.3851, lon=2.1734
- Seville: lat=37.3891, lon=-5.9845

WORKFLOW FOR WEATHER AND LOCATION QUESTIONS:

1. If asked about weather in a city NOT in the list above:
   - First use the get_location tool to search for information about that city
   - Look for latitude and longitude in the search results
   - Extract the coordinates from the search results

2. If asked directly about a location:
   - Use the get_location tool to search for information about that location
   - Return relevant information from the search results

3. For both cases:
   - Analyze the get_location results carefully
   - Look for text that mentions coordinates, latitude, longitude, "lat", "lon", etc.
   - Format your final answer clearly with the information found

You MUST use the get_location tool when asked about a location, providing:
- location: the location (string) to search for

Begin!

Question: {input}
Thought: {agent_scratchpad}
"""

llm = OllamaLLM(
   model="llama3", 
   base_url="http://localhost:11434", 
   temperature=0
)

prompt = PromptTemplate(
   template=template,
   input_variables=["input", "tools", "tool_names", "agent_scratchpad"], 
)

tools_for_agent = [
   StructuredTool.from_function(
      name="get_location",
      func=get_location,
      description="Get the location for a city.",
   )
]

agent = create_react_agent(
   llm=llm, 
   tools=tools_for_agent,
   prompt=prompt
)

agent_executor = AgentExecutor(
   agent=agent, 
   tools=tools_for_agent, 
   verbose=True,
)

# Primer agente - obtiene los datos de la localidad
weather_result = agent_executor.invoke(
   input={"input": "Quiero saber donde esta Santiago de Chile"}
)

print(weather_result["output"])

### **Weather Agent**

In [22]:
import requests

# Weather TOOL
def get_weather(input_str: str) -> dict:
   """
      Search for the current weather in a given city.
      
      Params:
         input_str (str): A string containing the coordinates either as JSON or in format "lat=40.4168, lon=-3.7038, part=''"
         
      Returns:
         dict: The weather data for the given city.
   """
   try:
      # Check if input is a JSON object (starts with { and ends with })
      if isinstance(input_str, dict):
         # It's already a dict
         params = input_str
      elif input_str.strip().startswith('{') and input_str.strip().endswith('}'):
         # Try to parse as JSON
         import json
         params = json.loads(input_str)
      else:
         # Parse the input string to extract parameters
         params = {}
         parts = input_str.split(',')
         
         for part in parts:
            key_value = part.strip().split('=')
            if len(key_value) == 2:
               key = key_value[0].strip()
               value = key_value[1].strip().strip("'\"")
               params[key] = value
      # city = params.get('city', '')
      lat = params.get('lat')
      lon = params.get('lon')
      part = params.get('part', '')
      units = "metric"  
      lang = "es"       
      print(f"Latitude: {lat}, Longitude: {lon}, Part: {part}")
      url = "https://api.openweathermap.org/data/2.5/weather"
      params = {
        # "q": city,
         "lat": lat,
         "lon": lon,
         "appid": WEATHER_API_KEY,
         "units": units,
         "lang": lang,
      }
      request = requests.get(url, params=params)
      request.raise_for_status()  # Raise an error for bad responsesk
      return request.json()
   except requests.exceptions.RequestException as e:
      print(f"Error fetching weather data: {e}")
      return {}
   except Exception as e:
      print(f"Error processing input: {e}")
      return {"error": f"Could not process input: {input_str}"}

In [23]:
from langchain_core.prompts import PromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.tools import StructuredTool
from langchain.agents import (create_react_agent, AgentExecutor)

weather_llm = OllamaLLM(model="llama3", base_url="http://localhost:11434", temperature=0)

template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

For weather questions about common cities, use these coordinates:
- Madrid: lat=40.4168, lon=-3.7038
- Barcelona: lat=41.3851, lon=2.1734
- Seville: lat=37.3891, lon=-5.9845

You MUST use the get_weather tool when asked about weather, providing:
- lat: the latitude (float)
- lon: the longitude (float)
- part: parts to exclude (leave as empty string "" for complete data)


When using the get_weather tool, you MUST format the Action Input as a JSON object with separate fields:
```json
  "lat": 40.4168,
  "lon": -3.7038,
  "part": ""
```
DO NOT use the format "lat=40.4168, lon=-3.7038, part=''".

### IMPORTANT
After you receive the weather data, DO NOT call get_weather again. 
Instead, analyze the data and provide a Final Answer based on it.

Begin!

Question: {input}
Thought: {agent_scratchpad}
"""

prompt = PromptTemplate(
   template=template,
   input_variables=["question", "tools", "tool_names", "agent_scratchpad"], 
)

tools_for_agent = [
   StructuredTool.from_function(
      name="get_weather",
      func=get_weather,
      description="Get the weather data for a city. Required parameters: lat (latitude), lon (longitude), part (parts to exclude).",
   )
]

weather_agent = create_react_agent(
   llm=weather_llm, 
   tools=tools_for_agent,
   prompt=prompt
)

weather_agent_executor = AgentExecutor(
   agent=weather_agent, 
   tools=tools_for_agent, 
   verbose=True,
)

   


### **INTERPRETER AGENT**

In [24]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """
Eres un asistente amable y útil que explica información meteorológica y financiera a los usuarios.

A continuación se te proporciona un JSON con datos o una respuesta de un agente:
```
{raw_response}
```

Tu trabajo es interpretar esta respuesta y explicarla de manera amigable y conversacional.

Si contiene datos meteorológicos, menciona:
- La temperatura actual 
- La descripción del clima (soleado, nublado, etc.)
- La humedad
- La velocidad del viento
- Cualquier otra información relevante

Si contiene datos financieros, menciona:
- El valor actual del indicador (dólar, UF, etc.)
- La variación respecto al día anterior
- La fecha de actualización
- Cualquier tendencia o información relevante

Si contiene datos de noticias, menciona:
- El título de la noticia
- Un resumen breve de la noticia

Si la respuesta contiene un error o no hay datos disponibles, explica el problema de manera amigable.

Responde SOLO con la interpretación, sin mencionar que estás interpretando datos o analizando un JSON.

Respuesta:
"""

interpreter_prompt = PromptTemplate(
   template=template,
   input_variables=["raw_response"],
)

llm_interpreter = OllamaLLM(
   model="llama3", 
   base_url="http://localhost:11434",
   temperature=0,
)
parser = StrOutputParser()
interpreter_chain = interpreter_prompt | llm_interpreter | parser


### **Playground** 

In [25]:

def query_weather(user_question):
   # Primer agente - obtiene los datos del clima
   weather_result = weather_agent_executor.invoke(
      input={"input": user_question}
   )
   
   # Segundo agente - interpreta y presenta los datos
   friendly_response = interpreter_chain.invoke(
      {"raw_response": weather_result["output"]}
   )
   
   return friendly_response


# Ejemplo de uso
user_query = "¿Qué tiempo hace hoy en Madrid?"
friendly_weather_response = query_weather(user_query)
print(f"Consulta: {user_query}\n\nRespuesta: {friendly_weather_response}")




> Entering new AgentExecutor chain...
Let's think about how to answer this question...

Action: get_weather
Action Input: {"lat": 40.4168, "lon": -3.7038, "part": ""}Latitude: 40.4168, Longitude: -3.7038, Part: 
{'coord': {'lon': -3.7038, 'lat': 40.4168}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'algo de nubes', 'icon': '02d'}], 'base': 'stations', 'main': {'temp': 15.46, 'feels_like': 14.5, 'temp_min': 15.12, 'temp_max': 16.53, 'pressure': 1015, 'humidity': 55, 'sea_level': 1015, 'grnd_level': 941}, 'visibility': 10000, 'wind': {'speed': 5.14, 'deg': 240}, 'clouds': {'all': 20}, 'dt': 1745261679, 'sys': {'type': 2, 'id': 2084029, 'country': 'ES', 'sunrise': 1745213240, 'sunset': 1745261957}, 'timezone': 7200, 'id': 6545083, 'name': 'Sol', 'cod': 200}Let's analyze the weather data!

From the observation, we can see that the current weather in Madrid is cloudy (main: 'Clouds', description: 'algo de nubes'). The temperature is around 15.46°C with a feels-like temperatur

In [28]:
def query_indicator(user_question):
   # Primer agente - obtiene los datos del indicador
   indicator_result = financial_agent_executor.invoke(
      input={"input": user_question}
   )
   
   # Segundo agente - interpreta y presenta los datos
   friendly_response = interpreter_chain.invoke(
      {"raw_response": indicator_result["output"]}
   )
   
   return friendly_response

# Ejemplo de uso
user_query = "¿Cuál es el valor del dólar hoy?"
friendly_indicator_response = query_indicator(user_query)
print(f"Consulta: {user_query}\n\nRespuesta: {friendly_indicator_response}")




> Entering new AgentExecutor chain...
Let's get started!

Thought: To find out the value of the dollar today, I need to use the `get_indicator` function and pass "dolar" as the indicator.

Action: get_indicator
Action Input: "dolar"

ConnectionError: HTTPSConnectionPool(host='mindicator.cl', port=443): Max retries exceeded with url: /api/dolar (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7d1b644ffa40>: Failed to resolve 'mindicator.cl' ([Errno -2] Name or service not known)"))